In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import re
import random
import cufflinks
pd.options.display.max_columns = 30
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='solar')

In [2]:
df = pd.read_csv('Travel_Data.csv', encoding="latin-1")
df.head()
print('We have', len(df), 'Travel places in the data')

We have 225 Travel places in the data


In [3]:
df.head()

,ID,Name,Type,Description
0,1,"Luksentyre, Outer Hebrides, Scotland",Beach,At the end of a winding road on the wind-batte...
1,2,"Dune Du Pilat, France",Beach,"In the warmer months, Europes tallest sand du..."
2,3,Ora Beach Indonesia,Beach,"With a higgledy-piggledy 37,000-mile coastline..."
3,4,Awaroa South Island New Zealand,Beach,Set deep in the heart of New Zealands gloriou...
4,5,Die Plast Walker Bay Nature Reserve South Africa,Beach,Dont be surprised to spot the occasional sout...


In [4]:
ex = df[df.index==0].values[0]
ex

array([1, 'Luksentyre, Outer Hebrides, Scotland', 'Beach',
       'At the end of a winding road on the wind-battered west coast of the Isle of Harris, you\x92ll find something to make your eyes pop. Luskentyre Beach is a paint splash of brilliant white, surrounded by almost impossibly azure water. On a sunny day, it\x92s so unerringly blue you may find it hard to believe you are at a beach in Scotland at all. But regardless of its paradoxically tropical feel, it\x92s at its most beautiful when eel-coloured storm clouds roll in and seas turn flinty, turning this dramatic landscape an eerie monochrome.'],
      dtype=object)

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
    
df['Description_Clean'] = df['Description'].apply(clean_text)

In [0]:
df.set_index('Name', inplace = True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['Description_Clean']) 
indices = pd.Series(df.index)

def get_recommendations(user_arr):
  rec_places = []
  vectors = []
  idx = []
  for u in user_arr:
    idx.append(indices[indices == u].index[0])
  for i in idx:
    vectors.append(tfidf_matrix[i])

  avg_vec = sum(vectors)/len(vectors)
  avg_vec_sim = linear_kernel(avg_vec, tfidf_matrix)

  avg_vec_sim = avg_vec_sim.flatten()

  top_10_rec = pd.Series(avg_vec_sim).sort_values(ascending = False)
  top_10_indexes = list(top_10_rec.iloc[1:len(user_arr)+11].index)
  rec_places = []
  for i in top_10_indexes:
        rec_places.append([list(df.index)[i], df['Type'][i]])

  return rec_places



In [0]:
user_arr = ['National Gallery of Canada in Ottawa'] 
rec_places = get_recommendations(user_arr)

In [0]:
rec_places = [i for i in rec_places if i[0] not in user_arr]

In [10]:
rec_places

[['The Art Institute of Chicago', 'Museums/Art Galleries'],
 ['State Hermitage Museum in St. Petersburg', 'Museums/Art Galleries'],
 ['The Musée du Louvre in Paris', 'Museums/Art Galleries'],
 ['The Metropolitan Museum Of Art in New York City', 'Museums/Art Galleries'],
 ['Mauritshuis in The Hague, Netherlands', 'Museums/Art Galleries'],
 ['The San Francisco Museum of Modern Art', 'Museums/Art Galleries'],
 ['Tate Modern in London', 'Museums/Art Galleries'],
 ['Museo Nacional Del Prado in Madrid', 'Museums/Art Galleries'],
 ['The National Palace Museum in Taipei', 'Museums/Art Galleries'],
 ['The British Museum in London', 'Museums/Art Galleries'],
 ['Vatican City', 'Historical Monument']]